In [1]:
import MetaTrader5 as mt
import pandas as pd
import plotly.express as px
import matplotlib.pylab as plt
import numpy as np
import talib
from talipp.indicators import EMA, SMA, Stoch, DPO
from joblib import dump
from datetime import datetime
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score, confusion_matrix, classification_report
from own_functions import *

from tsfresh import extract_features, select_features
from tsfresh.utilities.dataframe_functions import roll_time_series, make_forecasting_frame
from tsfresh.utilities.dataframe_functions import impute

In [2]:
mt.initialize()
login = 51708234
password ="4bM&wuVJcBTnjV"
server = "ICMarketsEU-Demo"
mt.login(login,password,server)

True

In [3]:
symbol="USDCAD"
timeframe=mt.TIMEFRAME_D1
#ohcl data
ohlc_data=pd.DataFrame(mt.copy_rates_range(symbol,timeframe,datetime(2010,1,1), datetime.now()))
ohlc_data['time'] = pd.to_datetime(ohlc_data['time'], unit='s')
df=pd.DataFrame(ohlc_data)[['time','open','high','low','close']]

#Indicators
df['WILLR_15']=talib.WILLR(df['high'],df['low'],df['open'],timeperiod=15) #Williams' %R
df['WILLR_23']=talib.WILLR(df['high'],df['low'],df['open'],timeperiod=23) #Williams' %R
df['WILLR_42']=talib.WILLR(df['high'],df['low'],df['open'],timeperiod=42) #Williams' %R
df['WILLR_145']=talib.WILLR(df['high'],df['low'],df['open'],timeperiod=145) #Williams' %R

#Buy & Sell Flags
df['b_flag'] = 0
df['s_flag'] = 0

df=df.dropna()
df=df.reset_index(drop=True)
""" csv_file_path = 'ICMT5USDCAD_2010_D1_1806.csv'  # Specify your desired path
df.to_csv(csv_file_path, index=False)
df """

" csv_file_path = 'ICMT5USDCAD_2010_D1_1806.csv'  # Specify your desired path\ndf.to_csv(csv_file_path, index=False)\ndf "

In [4]:
StopLoss=3
TakeProfit=4
BreakEvenRatio=StopLoss/(StopLoss+TakeProfit)
label_data(df,[StopLoss],[TakeProfit],80,symbol,False)
#print('BreatEvenRatio:', BreakEvenRatio)

Mean Candle: 0.008727453639634657


In [5]:
#target='b_flag'
target='s_flag'

selected_signal='WILLR_15'

df_melted = df[['time', selected_signal]].copy()
df_melted["Symbols"] = symbol

df_rolled = roll_time_series(df_melted, column_id="Symbols", column_sort="time",
                             max_timeshift=20, min_timeshift=5)

X = extract_features(df_rolled.drop("Symbols", axis=1), 
                     column_id="id", column_sort="time", column_value=selected_signal, 
                     impute_function=impute, show_warnings=False)

X = X.set_index(X.index.map(lambda x: x[1]), drop=True)
X.index.name = "time"
X=X.dropna()


selected_signal='WILLR_42'

df_melted = df[['time', selected_signal]].copy()
df_melted["Symbols"] = symbol

df_rolled = roll_time_series(df_melted, column_id="Symbols", column_sort="time",
                             max_timeshift=20, min_timeshift=5)

X3 = extract_features(df_rolled.drop("Symbols", axis=1), 
                     column_id="id", column_sort="time", column_value=selected_signal, 
                     impute_function=impute, show_warnings=False)

X3 = X3.set_index(X3.index.map(lambda x: x[1]), drop=True)
X3.index.name = "time"
X3=X3.dropna()


df=df.set_index('time')
df = df[df.index.isin(X.index)]

X = select_features(X, df[target])
X3 = select_features(X3, df[target])

df = pd.concat([X, X3, df], axis=1)
df=df.drop(columns=['high','low','close','open'])


X_selected = select_features(X, df[target])
X3_selected = select_features(X3, df[target])

# Get the list of selected feature names
selected_feature_names_X = list(X_selected.columns)
selected_feature_names_X3 = list(X3_selected.columns)

# Combine lists if you need a single list for all selected features
combined_selected_features = selected_feature_names_X + selected_feature_names_X3

print(combined_selected_features)

Feature Extraction: 100%|██████████| 30/30 [01:31<00:00,  3.05s/it]


['WILLR_15__change_quantiles__f_agg_"var"__isabs_False__qh_0.6__ql_0.0', 'WILLR_15__partial_autocorrelation__lag_4', 'WILLR_15__change_quantiles__f_agg_"mean"__isabs_True__qh_0.6__ql_0.0', 'WILLR_15__change_quantiles__f_agg_"mean"__isabs_True__qh_0.8__ql_0.0', 'WILLR_15__change_quantiles__f_agg_"var"__isabs_False__qh_0.8__ql_0.0', 'WILLR_15__partial_autocorrelation__lag_3', 'WILLR_15__change_quantiles__f_agg_"mean"__isabs_False__qh_0.8__ql_0.0', 'WILLR_15__lempel_ziv_complexity__bins_3', 'WILLR_15__change_quantiles__f_agg_"var"__isabs_False__qh_0.4__ql_0.0', 'WILLR_15__change_quantiles__f_agg_"var"__isabs_True__qh_0.6__ql_0.0', 'WILLR_15__change_quantiles__f_agg_"mean"__isabs_False__qh_1.0__ql_0.6', 'WILLR_15__permutation_entropy__dimension_7__tau_1', 'WILLR_15__change_quantiles__f_agg_"var"__isabs_True__qh_0.4__ql_0.0', 'WILLR_15__change_quantiles__f_agg_"mean"__isabs_True__qh_0.4__ql_0.0', 'WILLR_15__longest_strike_below_mean', 'WILLR_15__change_quantiles__f_agg_"var"__isabs_True__qh

In [6]:
""" sum_fp=0
sum_tp=0

split=int(0.90*len(df))

train_data, test_data = df.iloc[0:split], df.iloc[split+1:len(df)]

#Train data
x_train = train_data.iloc[:, 0:len(df.columns)-2].values
y_train = train_data.iloc[:, df.columns.get_loc(target)].values
#Test data
x_test = test_data.iloc[:, 0:len(df.columns)-2].values
y_test = test_data.iloc[:, df.columns.get_loc(target)].values


#Scale Data
sc_mt=StandardScaler()

x_train=sc_mt.fit_transform(x_train)

x_test=sc_mt.transform(x_test)

#dump(sc_mt,'Dump_USDCAD_H4Sell\scaler.joblib')

# Hyperparameter
n_estimators = 100
class_weight = {0: 15, 1: 6}
max_features = 'log2'
random_state = 1

# Initialise RandomForestClassifier
rf_classifier_mt = RandomForestClassifier(
    n_estimators=n_estimators,
    class_weight=class_weight,
    max_features=max_features,
    random_state=random_state
)

# Train Modell
rf_classifier_mt.fit(x_train, y_train)

#dump(rf_classifier_mt,'Dump_USDCAD_H4Sell\model.joblib')

# Predict
y_pred = rf_classifier_mt.predict(x_test)

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

false_positives = confusion_matrix(y_test, y_pred)[0][1]
true_positives = confusion_matrix(y_test, y_pred)[1][1]

sum_fp=sum_fp+false_positives
sum_tp=sum_tp+true_positives

#print("\nClassification Report:")
#print(classification_report(y_test, y_pred))

precision = precision_score(y_test, y_pred)
#print("\nPrecision Score:", round(precision,2))
#print('BreakEvenRatio:', round(BreakEvenRatio,2))
#print('\n')
print('WIN/LOSS-Diff:', round(100*(precision-BreakEvenRatio),2), '%')
print('sum_fp:', sum_fp)
print('sum_tp:', sum_tp)
#print('Diff:', sum_tp-sum_fp)
print('precision:',precision)
print('Ratio total:', round(100*(sum_tp/(sum_fp+sum_tp)) ,2))
print('BreakEvenRatio:', round(BreakEvenRatio,2))
print('____________________________________________________________________________________________________________________________') """

' sum_fp=0\nsum_tp=0\n\nsplit=int(0.90*len(df))\n\ntrain_data, test_data = df.iloc[0:split], df.iloc[split+1:len(df)]\n\n#Train data\nx_train = train_data.iloc[:, 0:len(df.columns)-2].values\ny_train = train_data.iloc[:, df.columns.get_loc(target)].values\n#Test data\nx_test = test_data.iloc[:, 0:len(df.columns)-2].values\ny_test = test_data.iloc[:, df.columns.get_loc(target)].values\n\n\n#Scale Data\nsc_mt=StandardScaler()\n\nx_train=sc_mt.fit_transform(x_train)\n\nx_test=sc_mt.transform(x_test)\n\n#dump(sc_mt,\'Dump_USDCAD_H4Sell\\scaler.joblib\')\n\n# Hyperparameter\nn_estimators = 100\nclass_weight = {0: 15, 1: 6}\nmax_features = \'log2\'\nrandom_state = 1\n\n# Initialise RandomForestClassifier\nrf_classifier_mt = RandomForestClassifier(\n    n_estimators=n_estimators,\n    class_weight=class_weight,\n    max_features=max_features,\n    random_state=random_state\n)\n\n# Train Modell\nrf_classifier_mt.fit(x_train, y_train)\n\n#dump(rf_classifier_mt,\'Dump_USDCAD_H4Sell\\model.joblib

In [7]:
""" df.index = pd.to_datetime(df.index, unit='s')
df.index = df.index.strftime('%Y-%m-%d %H:%M:%S')
df_pred = pd.DataFrame(index=df.iloc[split+1:len(df)].index)
df_pred['prediction'] = y_pred
df_pred.to_csv('predictUSDCAD_D1Sell.csv') """

" df.index = pd.to_datetime(df.index, unit='s')\ndf.index = df.index.strftime('%Y-%m-%d %H:%M:%S')\ndf_pred = pd.DataFrame(index=df.iloc[split+1:len(df)].index)\ndf_pred['prediction'] = y_pred\ndf_pred.to_csv('predictUSDCAD_D1Sell.csv') "

In [8]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, precision_score
from joblib import dump
from datetime import datetime
import pytz

# Set timezone to UTC
utc_tz = pytz.utc


# Initialize counters
sum_fp = 0
sum_tp = 0

# Split data
split = int(0.90 * len(df))
train_data, test_data = df.iloc[0:split], df.iloc[split+1:len(df)]

# Train data
x_train = train_data.iloc[:, 0:len(df.columns)-2].values
y_train = train_data.iloc[:, df.columns.get_loc(target)].values

# Test data
x_test = test_data.iloc[:, 0:len(df.columns)-2].values
y_test = test_data.iloc[:, df.columns.get_loc(target)].values

# Scale Data
sc_mt = StandardScaler()
x_train = sc_mt.fit_transform(x_train)
x_test = sc_mt.transform(x_test)
dump(sc_mt, 'AdaBoost_USDCAD_D1Sell/scaler.joblib')

# Hyperparameters
n_estimators = 80
class_weight = {0: 1, 1: 15}
max_features = 'log2'
random_state = 1

# Initialize RandomForestClassifier
rf_classifier_mt = RandomForestClassifier(
    n_estimators=n_estimators,
    class_weight=class_weight,
    max_features=max_features,
    random_state=random_state
)

# Initialize AdaBoostClassifier with RandomForest as the base estimator
ada_classifier_mt = AdaBoostClassifier(
    base_estimator=rf_classifier_mt,
    n_estimators=80,
    learning_rate=1.0,
    random_state=random_state
)

# Train Model
ada_classifier_mt.fit(x_train, y_train)
dump(ada_classifier_mt, 'AdaBoost_USDCAD_D1Sell/model.joblib')

# Predict
y_pred = ada_classifier_mt.predict(x_test)

# Metrics Calculation
print("Confusion Matrix:")
cm = confusion_matrix(y_test, y_pred)
print(cm)

false_positives = cm[0][1]
true_positives = cm[1][1]

sum_fp += false_positives
sum_tp += true_positives

precision = precision_score(y_test, y_pred)

print("\nMetrics:")
print(f"WIN/LOSS-Diff: {round(100*(precision-BreakEvenRatio), 2)} %")
print(f"sum_fp: {sum_fp}")
print(f"sum_tp: {sum_tp}")
print(f"precision: {precision}")
print(f"Ratio total: {round(100*(sum_tp / (sum_fp + sum_tp)), 2)}")
print(f"BreakEvenRatio: {round(BreakEvenRatio, 2)}")
print('____________________________________________________________________________________________________________________________')

Confusion Matrix:
[[226  29]
 [ 85  20]]

Metrics:
WIN/LOSS-Diff: -2.04 %
sum_fp: 29
sum_tp: 20
precision: 0.40816326530612246
Ratio total: 40.82
BreakEvenRatio: 0.43
____________________________________________________________________________________________________________________________


In [9]:
#df.index = pd.to_datetime(df.index, unit='s')
df.index = df.index.strftime('%Y-%m-%d %H:%M:%S')
df_pred = pd.DataFrame(index=df.iloc[split+1:len(df)].index)
df_pred['prediction'] = y_pred
df_pred.to_csv('predictadaboostUSDCAD_D1Sell.csv')#